Firstly, I needed to import all of the necessary PyQt libraries and modules for the basis of this Python application project.

In [ ]:
import json
import sys
import time
import datetime
import psutil
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QDateTimeEdit, QCalendarWidget,QMenu, QSystemTrayIcon, QComboBox, QDialog, QCheckBox, QMainWindow, QApplication, QWidget, QVBoxLayout, QSizePolicy, QLabel, QTimeEdit, QLineEdit, QPushButton, QTextEdit, QMessageBox, QAction, QToolBar
from PyQt5.QtCore import QThread, pyqtSignal, Qt, QTime, QTimer, QSize, QDateTime, QDate
from PyQt5.QtGui import QIcon, QKeySequence, QFont
import os

To style specific windows and buttons within the application in a way that I felt was user friendly for , I added these style specific variables.

Calendar Style Sheet:

In [ ]:
"""

    /* Style for spinbox buttons */
    QCalendarWidget QSpinBox::up-button, QCalendarWidget QSpinBox::down-button {
        subcontrol-origin: border;
        subcontrol-position: center right;
        border: none;
    }
    QCalendarWidget QSpinBox::up-arrow, QCalendarWidget QSpinBox::down-arrow {
        width: 10px;
        height: 10px;
    }

    /* Style for month and year buttons */
    QCalendarWidget QWidget#qt_calendar_monthbutton,
    QCalendarWidget QWidget#qt_calendar_yearbutton {
        border-top-left-radius: 3px; 
        border-top-right-radius: 3px; 
        border-bottom-left-radius: 3px; 
        border-bottom-right-radius: 3px;
    }

    /* Hide the menu indicator */
    QCalendarWidget QToolButton::menu-indicator {
        image: none;
    }
    
    /* Style for the navigation bar */
    QCalendarWidget #qt_calendar_navigationbar {
        background-color: #f0f0f0;  /* Change as needed */
    }

    /* Style for the calendar view */
    QCalendarWidget #qt_calendar_calendarview {
        selection-background-color: #4285f4;  /* Blue color for selected dates */
        selection-color: white;  /* White text for selected dates */
    }

    /* Style for highlighting the current date */
    QCalendarWidget QAbstractItemView {
        selection-background-color: #ff4500; /* Orange color for current date */
        selection-color: white;  /* White text for current date */
    }

"""

Text Style:

In [ ]:
"""
    QLabel {
        background-color: #333333;  # Dark background
        color: #ffffff;             # White text
        padding: 5px;               # Padding around the text
        border-radius: 5px;         # Rounded corners
    }
"""

APPLICATION_BUTTONS_STYLESHEET:

In [ ]:
"""
    QPushButton {
        color: black;
        background-color: #f0f0f0; 
        border-top-left-radius: 3px; 
        border-top-right-radius: 3px; 
        border-bottom-left-radius: 3px; 
        border-bottom-right-radius: 3px; 
        border: 1px solid #c0c0c0;
        min-width: 80px;    /* Minimum width */
        min-height: 55px;   /* Minimum height */
    }
    QPushButton:hover {
        background-color: #e0e0e0; 
        border: 1px solid #d0d0d0; 
    }
    QPushButton:pressed {
        background-color: #d0d0d0;
    }
"""

PROMPT_BUTTON_STYLESHEET:

In [ ]:
"""
    QPushButton {
        color: black;
        background-color: #fdfdfd; 
        border-top-left-radius: 3px; 
        border-top-right-radius: 3px; 
        border-bottom-left-radius: 3px; 
        border-bottom-right-radius: 3px; 
        border: 1px solid #bababa;
        min-width: 80px;    /* Minimum width */
        min-height: 30px;   /* Minimum height */
        font-size: 15px;    /* Font size */
    }
    QPushButton:hover {
        background-color: #e0eef9; 
        border: 1px solid #217cc4; 
    }
    QPushButton:pressed {
        background-color: #cce4f7;
    }
"""

APPLY_BUTTON_STYLESHEET:

In [ ]:
"""
    QPushButton { 
        background-color: #fdfdfd;
        border-top-left-radius: 3px; 
        border-top-right-radius: 3px; 
        border-bottom-left-radius: 3px; 
        border-bottom-right-radius: 3px; 
        border: 1px solid #bababa;
        min-width: 30px;    /* Minimum width */
        min-height: 40px;   /* Minimum height */
        color: black;  /* Set text color to black */
    }
    QPushButton:hover { 
        background-color: #e0eef9;
        border: 1px solid #217cc4;
    }
    QPushButton:pressed { 
        background-color: #cce4f7;
    }
"""

BUTTON_STYLESHEET:

In [ ]:
"""
    QPushButton { 
        background-color: #fdfdfd;
        border-top-left-radius: 3px; 
        border-top-right-radius: 3px; 
        border-bottom-left-radius: 3px; 
        border-bottom-right-radius: 3px; 
        border: 1px solid #bababa;
        min-width: 30px;    /* Minimum width */
        min-height: 40px;   /* Minimum height */
        color: black;  /* Set text color to black */
    }
    QPushButton:hover { 
        background-color: #e0eef9;
        border: 1px solid #217cc4;
    }
    QPushButton:pressed { 
        background-color: #cce4f7;
    }
"""

For this apps secondary functionality, I wanted to include a feature which has been useful for myself which incolves settting up a background process monitoring function to listen for the launch of the specific applications listed.

In [ ]:
class ProcessMonitorThread(QThread):
    program_launched = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.excel_running = False
        self.powerpoint_running = False
        self.word_running = False
        self.outlook_running = False
        self.teams_running = False
        self.vscode_running = False
        self.android_studio_running = False
        self.dbeaver_running = False
        self.chrome_running = False
        self.edge_running = False
        self.firefox_running = False
        

    def run(self):
        while True:
            excel_current_state = "EXCEL.EXE" in (p.name() for p in psutil.process_iter())
            powerpoint_current_state = "POWERPNT.EXE" in (p.name() for p in psutil.process_iter())
            word_current_state = "WINWORD.EXE" in (p.name() for p in psutil.process_iter())
            outlook_current_state = "OUTLOOK.EXE" in (p.name() for p in psutil.process_iter())
            teams_current_state = "Teams.exe" in (p.name() for p in psutil.process_iter())
            vscode_current_state = "devenv.exe" in (p.name() for p in psutil.process_iter())
            android_studio_current_state = "studio64.exe" in (p.name() for p in psutil.process_iter())
            dbeaver_current_state = "dbeaver.exe" in (p.name().lower() for p in psutil.process_iter())
            chrome_current_state = "chrome.exe" in (p.name().lower() for p in psutil.process_iter())
            edge_current_state = "msedge.exe" in (p.name().lower() for p in psutil.process_iter())
            firefox_current_state = "firefox.exe" in (p.name().lower() for p in psutil.process_iter())


            
            if excel_current_state and not self.excel_running:
                self.program_launched.emit("excel")
            self.excel_running = excel_current_state

            
            if powerpoint_current_state and not self.powerpoint_running:
                self.program_launched.emit("powerpoint")
            self.powerpoint_running = powerpoint_current_state



            if word_current_state and not self.word_running:
                self.program_launched.emit("word")
            self.word_running = word_current_state

            
            if outlook_current_state and not self.outlook_running:
                self.program_launched.emit("outlook")
            self.outlook_running = outlook_current_state
            
            
            
            if teams_current_state and not self.teams_running:
                self.program_launched.emit("teams")
            self.teams_running = teams_current_state

            
            if vscode_current_state and not self.vscode_running:
                self.program_launched.emit("vscode")
            self.vscode_running = vscode_current_state
            

            
            if android_studio_current_state and not self.android_studio_running:
                self.program_launched.emit("android studio")
            self.android_studio_running = android_studio_current_state
            
            
            
            if dbeaver_current_state and not self.dbeaver_running:
                self.program_launched.emit("dbeaver")
            self.dbeaver_running = dbeaver_current_state
            
            
            
            if chrome_current_state and not self.chrome_running:
                self.program_launched.emit("chrome")
            self.chrome_running = chrome_current_state
            
            
            
            if edge_current_state and not self.edge_running:
                self.program_launched.emit("edge")
            self.edge_running = edge_current_state

        
            
            if firefox_current_state and not self.firefox_running:
                self.program_launched.emit("firefox")
            self.firefox_running = firefox_current_state



            time.sleep(1)

This next part describes the main window of the application which hosts as the parent for the other windows that depend on the users interactions with the calendar button, or the application buttons.

In [ ]:



        

    def CreateMainWindowText(self):
        shortcut_info_label = QLabel(self)
        shortcut_info_label.setText("Press <b>Ctrl+Shift+A</b> to open preferences window")
        shortcut_info_label.setAlignment(Qt.AlignCenter)  # Align the text to the center
        shortcut_info_label.setFont(QFont("Arial", 10))  # Set font and size
        # shortcut_info_label.setStyleSheet(TEXT_STYLE)
        return shortcut_info_label
    
    



        
        
    def initAttributes(self): 
        self.currentTheme = ""
        self.notes = {"excel": "", "powerpoint": "", "word": "", "outlook": "", "teams": "", "vscode": "", "android studio": "", "dbeaver": "", "chrome": "", "edge": "", "firefox": "" }  
        self.noteText = "" 
        self.currentMode = "light"  
        self.delay_time = datetime.timedelta(seconds=0)
        
        
    def startBackgroundProcesses(self):
        self.monitor_thread = ProcessMonitorThread()
        self.monitor_thread.program_launched.connect(self.onApplicationLaunched)
        self.monitor_thread.start()
        
        
        
    def setupWindowProperties(self):
        self.setWindowTitle('EventNotes')
        self.setWindowFlags(self.windowFlags() & ~Qt.WindowMaximizeButtonHint)
        self.adjustMainWindowHeightDynamically()
        


    def adjustMainWindowHeightDynamically(self):
        # Adjust the height based on the content
        requiredHeight = max(self.sizeHint().height(), 150) # Ensure a minimum height of 150t
        self.setFixedWidth(500)
        self.setMaximumHeight(requiredHeight)
        




    def initToolbar(self, layout):
        toolbar = QToolBar("Main Toolbar")
        self.addToolBar(Qt.TopToolBarArea, toolbar)
        self.themeColorButton = ThemeColorButton(self, self.toggleMode)
        toolbar.addWidget(self.themeColorButton)
        
        

        self.preferencesButton = PreferencesButton(self, self.openPreferences)
        toolbar.addWidget(self.preferencesButton)
        self.PreferencesWindow = PreferencesWindow(self)
        

        spacer = QWidget()
        spacer.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Preferred)
        toolbar.addWidget(spacer)
        

        self.helpButton = HelpButton(self, self.HelpMessageWindow)
        toolbar.addWidget(self.helpButton)
        


    def initApplicationButtons(self, layout):
        self.initExcelButton(layout)
        self.initPowerPointButton(layout)
        self.initWordButton(layout)
        self.initOutlookButton(layout)
        self.initTeamsButton(layout)
        self.initVscodeButton(layout)
        self.initAndroidstudioButton(layout)
        self.initDbeaverButton(layout)
        self.initChromeButton(layout)
        self.initEdgeButton(layout)
        self.initFirefoxButton(layout)






################################ Calendar Notes Handling ##########################################

    def handleCalendarNote(self, note_text, selected_datetime):
        # Handle the confirmed note and datetime here
        self.scheduledNote = (selected_datetime, note_text)
        self.save_scheduled_note()
        self.recalculateTimerIfNeeded(selected_datetime)
        
    def save_scheduled_note(self):
        
        # Determine the full path for the calendar_notes directory
        calendar_notes_dir = os.path.join(get_base_path(), 'calendar_notes')
        
        # Create the calendar_notes directory if it does not exist
        if not os.path.exists(calendar_notes_dir):
            os.makedirs(calendar_notes_dir)
        # Save the single note to a file
        with open(os.path.join(get_base_path(), 'calendar_notes/scheduled_note.json'), 'w') as file:
            note_time, note_text = self.scheduledNote
            json.dump({'note_time': note_time.isoformat(), 'note_text': note_text}, file)

            
            
            
    def load_scheduled_note(self):
        current_hours_time = datetime.datetime.now().time().replace(minute=0, second=0, microsecond=0)
        try:
            with open(os.path.join(get_base_path(), 'calendar_notes/scheduled_note.json'), 'r') as file:
                data = json.load(file)
                scheduled_datetime = datetime.datetime.fromisoformat(data['note_time'])
                self.scheduledNote = (scheduled_datetime, data['note_text'])

                
                # Check if the note is due today and set up the timer
                if scheduled_datetime.date() == datetime.datetime.now().date() and current_hours_time >= scheduled_datetime.time():
                    self.openReminderWindowForCalendar(self.scheduledNote[1])
                elif scheduled_datetime.date() == datetime.datetime.now().date():
                    self.setupHourlyCheck()

        except (FileNotFoundError, ValueError):
            # Handle file not found or invalid content
            self.scheduledNote = None
            
            
            
            
            
    def recalculateTimerIfNeeded(self, note_time):
        # Only recalculate the timer if the note is for today
        if note_time.date() == datetime.datetime.now().date():
            self.setupHourlyCheck()
            
            

    
    def checkForDueNote(self):
        # Check if the scheduled note is due today
        if self.scheduledNote:
            note_time, _ = self.scheduledNote
            if note_time.date() == datetime.datetime.now().date():
                self.setupHourlyCheck()
                
                

    def setupHourlyCheck(self):
            # If the timer is not already running, set it up
            if not hasattr(self, 'hourlyCheckTimer') or not self.hourlyCheckTimer.isActive():
                self.hourlyCheckTimer = QTimer(self)
                self.hourlyCheckTimer.timeout.connect(self.checkHourlyNotes)
                # Check every hour starting from the next full hour
                millis_till_next_hour = (60 - datetime.datetime.now().minute) * 60000
                self.hourlyCheckTimer.start(millis_till_next_hour)
            

    def checkHourlyNotes(self):
           current_time = datetime.datetime.now().replace(minute=0, second=0, microsecond=0)
           if self.scheduledNote:
               note_time, note_text = self.scheduledNote
               if current_time >= note_time:
                   self.openReminderWindowForCalendar(note_text)
                   self.scheduledNote = None  # Clear the scheduled note after displaying the reminder




    def openReminderWindowForCalendar(self, note_text):
        # Use your existing method to display the note
        self.ReminderWindow = ReminderWindow(note_text, self.geometry())
        self.ReminderWindow.setStyleSheet(self.currentTheme)
        self.ReminderWindow.show()

################################ Calendar Notes Handling ##########################################



    def initExcelButton(self, layout):
        self.excelButton = QPushButton(self)
        self.excelButton.hide()
        excelIcon = QIcon(icon_path('excel_icon.png'))
        self.excelButton.setIcon(excelIcon)
        self.excelButton.setIconSize(QSize(48, 48))
        self.excelButton.clicked.connect(self.openNotebookWindowForExcel)

        self.excelButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.excelButton)
        
    def initPowerPointButton(self, layout):
        self.powerPointButton = QPushButton(self)
        self.powerPointButton.hide()
        powerPointButton = QIcon(icon_path('powerp_icon.png'))
        self.powerPointButton.setIcon(powerPointButton)
        self.powerPointButton.setIconSize(QSize(48, 48))
        self.powerPointButton.clicked.connect(self.openNotebookWindowForPowerPoint)
        self.powerPointButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.powerPointButton)
        
        
    def initWordButton(self, layout):
        self.wordButton = QPushButton(self)
        self.wordButton.hide()
        wordButton = QIcon(icon_path('word_icon.png'))
        self.wordButton.setIcon(wordButton)
        self.wordButton.setIconSize(QSize(48, 48))
        self.wordButton.clicked.connect(self.openNotebookWindowForWord)
        self.wordButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.wordButton)
        
    def initOutlookButton(self, layout):
        self.outlookButton = QPushButton(self)
        self.outlookButton.hide()
        outlookButton = QIcon(icon_path('outlook_icon.png'))
        self.outlookButton.setIcon(outlookButton)
        self.outlookButton.setIconSize(QSize(48, 48))
        self.outlookButton.clicked.connect(self.openNotebookWindowForOutlook)
        self.outlookButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.outlookButton)
        
    def initTeamsButton(self, layout):
        self.teamsButton = QPushButton(self)
        self.teamsButton.hide()
        teamsButton = QIcon(icon_path('teams_icon.png'))
        self.teamsButton.setIcon(teamsButton)
        self.teamsButton.setIconSize(QSize(48, 48))
        self.teamsButton.clicked.connect(self.openNotebookWindowForTeams)
        self.teamsButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.teamsButton)
        
    def initVscodeButton(self, layout):
        self.vscodeButton = QPushButton(self)
        self.vscodeButton.hide()
        vscodeButton = QIcon(icon_path('vscode_icon.png'))
        self.vscodeButton.setIcon(vscodeButton)
        self.vscodeButton.setIconSize(QSize(48, 48))
        self.vscodeButton.clicked.connect(self.openNotebookWindowForVscode)
        self.vscodeButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.vscodeButton)
        
    def initAndroidstudioButton(self, layout):
        self.androidstudioButton = QPushButton(self)
        self.androidstudioButton.hide()
        androidstudioButton = QIcon(icon_path('android_studio_icon.png'))
        self.androidstudioButton.setIcon(androidstudioButton)
        self.androidstudioButton.setIconSize(QSize(48, 48))
        self.androidstudioButton.clicked.connect(self.openNotebookWindowForAndroidstudio)
        self.androidstudioButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.androidstudioButton)
        
        
    def initDbeaverButton(self, layout):
        self.dbeaverButton = QPushButton(self)
        self.dbeaverButton.hide()
        dbeaverButton = QIcon(icon_path('dbeaver_icon.png'))
        self.dbeaverButton.setIcon(dbeaverButton)
        self.dbeaverButton.setIconSize(QSize(48, 48))
        self.dbeaverButton.clicked.connect(self.openNotebookWindowForDbeaver)
        self.dbeaverButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.dbeaverButton)
        
        
    def initChromeButton(self, layout):
        self.chromeButton = QPushButton(self)
        self.chromeButton.hide()
        chromeButton = QIcon(icon_path('chrome_icon.png'))
        self.chromeButton.setIcon(chromeButton)
        self.chromeButton.setIconSize(QSize(48, 48))
        self.chromeButton.clicked.connect(self.openNotebookWindowForChrome)
        self.chromeButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.chromeButton)
        
        
    def initEdgeButton(self, layout):
        self.edgeButton = QPushButton(self)
        self.edgeButton.hide()
        edgeButton = QIcon(icon_path('edge_icon.png'))
        self.edgeButton.setIcon(edgeButton)
        self.edgeButton.setIconSize(QSize(48, 48))
        self.edgeButton.clicked.connect(self.openNotebookWindowForEdge)
        self.edgeButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.edgeButton)
        
        
    def initFirefoxButton(self, layout):
        self.firefoxButton = QPushButton(self)
        self.firefoxButton.hide()
        firefoxButton = QIcon(icon_path('firefox_icon.png'))
        self.firefoxButton.setIcon(firefoxButton)
        self.firefoxButton.setIconSize(QSize(48, 48))
        self.firefoxButton.clicked.connect(self.openNotebookWindowForFirefox)
        self.firefoxButton.setStyleSheet(APPLICATION_BUTTONS_STYLESHEET)
        layout.addWidget(self.firefoxButton)


        
        
        
        
    def openPreferences(self):
        if self.PreferencesWindow.exec_():
            preferences = self.PreferencesWindow.getPreferences()
            self.applyPreferences(preferences)



    def applyPreferences(self, preferences):
        # Update button visibility based on preferences
        self.excelButton.setVisible(preferences.get('excel', False))
        self.powerPointButton.setVisible(preferences.get('powerpoint', False))
        self.wordButton.setVisible(preferences.get('word', False))
        self.outlookButton.setVisible(preferences.get('outlook', False))
        self.teamsButton.setVisible(preferences.get('teams', False))
        self.vscodeButton.setVisible(preferences.get('vscode', False))
        self.androidstudioButton.setVisible(preferences.get('android studio', False))
        self.dbeaverButton.setVisible(preferences.get('dbeaver', False))
        self.chromeButton.setVisible(preferences.get('chrome', False))
        self.edgeButton.setVisible(preferences.get('edge', False))
        self.firefoxButton.setVisible(preferences.get('firefox', False))

        self.adjustMainWindowHeightDynamically()
   

        
        # self.resize(currentSize.width(), newHeight)
        AppButtonVisible = any(preferences.values())  # Check if any application button is visible
        self.shortcut_info_label.setVisible(not AppButtonVisible)
        
        if not AppButtonVisible:
            # If no applications are selected, revert to default window size
            self.resize(self.defaultWindowSize)
            pass


    def setLightMode(self):
        self.currentTheme = "QWidget { background-color: white; color: black; }"
        self.setStyleSheet(self.currentTheme)


    def setDarkMode(self):
        self.currentTheme = "QWidget { background-color: #40414f; color: white; }"
        self.setStyleSheet(self.currentTheme)








        
        
    def updateChildWindowStyles(self):
        if self.notebookWindow:
            self.notebookWindow.setStyleSheet(self.currentTheme)
        if self.ReminderWindow:
            self.ReminderWindow.setStyleSheet(self.currentTheme)
        if self.HelpMessageWindow:
            self.HelpMessageWindow.setStyleSheet(self.currentTheme)
        if self.closeEvent:
            self.closeEvent.setStyleSheet(self.currentTheme)
        if self.CalendarNoteWindow:
            self.CalendarNoteWindow.setStyleSheet(self.currentTheme)







    def toggleMode(self):
        if self.currentMode == "light":
            self.setDarkMode()
            self.currentMode = "dark"
            self.themeColorButton.setIcon(self.themeColorButton.sunIcon) 
            self.preferencesButton.setIcon(self.preferencesButton.preferences_icon_dm)
            self.preferencesButton.isDarkMode = True
        else:
            self.setLightMode()
            self.currentMode = "light"
            self.themeColorButton.setIcon(self.themeColorButton.crescentIcon)
            self.preferencesButton.setIcon(self.preferencesButton.preferences_icon_lm)
            self.preferencesButton.isDarkMode = False








    def createThemedMessageBox(self, title, title_icon, text, icon, buttons, defaultButton, customSize=None):
        msgBox = QMessageBox()
        msgBox.setWindowTitle(title)
        msgBox.setWindowIcon(title_icon)
        msgBox.setText(text)
        msgBox.setIcon(icon)
        msgBox.setStandardButtons(buttons)
        msgBox.setDefaultButton(defaultButton)
        msgBox.setStyleSheet(PROMPT_BUTTON_STYLESHEET)
        msgBox.setStyleSheet(self.currentTheme)
        if customSize:
            msgBox.setSizeGripEnabled(True)
            QSpacerItem = QtWidgets.QSpacerItem(customSize[0], customSize[1], QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding)
            layout = msgBox.layout()
            layout.addItem(QSpacerItem, layout.rowCount(), 0, 1, layout.columnCount())
        return msgBox  
        
        
    
    
    def HelpMessageWindow(self):
        customSize = (800, 20)  # Width, Height
        msgBox = self.createThemedMessageBox("Help",
                                             QIcon(icon_path('help_icon.png')),
                                            "<html>"
                                            "<body>"
                                            "<b>Application Program Event Notes Disclaimers:</b><br>"
                                            "In the preferences window, select the checkbox for the application(s) you wish to keep displayed in the main application window to create an Event Note from.<br><br>"
                                            "Updates made to an Event Note for actively running programs will display once the associated program has been terminated and relaunched.<br><br>"
                                            "The application program associated with a newly created Event Note must remain active after it has been confirmed for the note to be successfully displayed.<br><br>"
                                            "<b>Calendar Event Notes Disclaimers:</b><br>"
                                            "Calendar event notes must be created for a specific time in the future, otherwise the Event Note will not be displayed.<br><br>"
                                            "If you reboot or restart your device, the Calendar Event Note will only be displayed after this application has been relaunched, therefore we recommend to add this app to the 'lauch on start up' list via your windows settings.<br><br>"
                                            "<b>Event Notes: Basic and Premium</b><br>"
                                            "The basic version of this application will be limited to hour only based Calendar Event Notes. The premium version of this application allows users to edit the hour and minute timer conditions of the Calendar Event Notes.<br><br>"
                                            "<b>Privacy Statement:</b><br>"
                                            "To achieve full functionality of the Calendar Event Notes, the contents (date,time and notes)  of any created Calendar Event Notes are saved to a JSON file - 'scheduled_notes', within the folder - 'calendar_notes' of this applications directory, and the contents will be overwritten with each new Calendar Event Note.<br><br>"
                                            "Windows application associated Event Notes are not saved to a physical file on your machine.<br><br>"
                                            "<b>Rate our App:</b><br>"
                                            "If you find this application useful, consider leaving a rating via the Microsoft App Store and if you really like this application, tell us your thoughts in the review section."
                                            "</body>"
                                            "</html>",
                                              QMessageBox.Information,
                                              QMessageBox.Ok,
                                              QMessageBox.Ok,
                                              customSize=customSize)
        
        msgBox.exec_()
        
        
    def closeEvent(self, event):
        msgBox = self.createThemedMessageBox(" ",
                                              QIcon(icon_path('transparent_icon.png')),
                                             "Your preferences and EventNotes will be lost.\n\n"
                                             "Are you sure you want to exit?" ,
                                              QMessageBox.Warning,
                                              QMessageBox.Yes | QMessageBox.No, 
                                              QMessageBox.No)
        
        reply = msgBox.exec_()
        if reply == QMessageBox.Yes:
            
            
            event.accept()  
        else:
            
            event.ignore()  







    def openCalendarNoteWindow(self):
        # QMessageBox.information(self, "Test", "Calendar button clicked")
        try:
            self.calendarNoteWindow = CalendarNoteWindow(parent=self, callback=self.handleCalendarNote, currentTheme=self.currentTheme)
            self.calendarNoteWindow.setStyleSheet(self.currentTheme)
            self.calendarNoteWindow.show()
        except Exception as e:
            print("Error opening CalendarNoteWindow:", e)




    def openNotebookWindowForExcel(self):
        self.notebookWindow = NotebookWindow("excel", callback=lambda note: self.receiveNoteText(note, "excel"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()
        
        

    def openNotebookWindowForPowerPoint(self):
        self.notebookWindow = NotebookWindow("powerpoint", callback=lambda note: self.receiveNoteText(note, "powerpoint"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()
        
    
    
    def openNotebookWindowForWord(self):
        self.notebookWindow = NotebookWindow("word", callback=lambda note: self.receiveNoteText(note, "word"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()
        
        
        

    def openNotebookWindowForOutlook(self):
        self.notebookWindow = NotebookWindow("outlook", callback=lambda note: self.receiveNoteText(note, "outlook"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()



    def openNotebookWindowForTeams(self):
        self.notebookWindow = NotebookWindow("teams", callback=lambda note: self.receiveNoteText(note, "teams"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()



    def openNotebookWindowForVscode(self):
        self.notebookWindow = NotebookWindow("vscode", callback=lambda note: self.receiveNoteText(note, "vscode"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()
        
        
        
    def openNotebookWindowForAndroidstudio(self):
        self.notebookWindow = NotebookWindow("android Studio", callback=lambda note: self.receiveNoteText(note, "android studio"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()
        

    def openNotebookWindowForDbeaver(self):
        self.notebookWindow = NotebookWindow("dbeaver", callback=lambda note: self.receiveNoteText(note, "dbeaver"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()



    def openNotebookWindowForChrome(self):
        self.notebookWindow = NotebookWindow("chrome", callback=lambda note: self.receiveNoteText(note, "chrome"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()



    def openNotebookWindowForEdge(self):
        self.notebookWindow = NotebookWindow("edge", callback=lambda note: self.receiveNoteText(note, "edge"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()
        
        
        
    def openNotebookWindowForFirefox(self):
        self.notebookWindow = NotebookWindow("firefox", callback=lambda note: self.receiveNoteText(note, "firefox"), delayCallback=self.setDelay)
        self.notebookWindow.setStyleSheet(self.currentTheme)
        self.notebookWindow.show()
        
        
    def receiveNoteText(self, note_text, app_name):
        self.notes[app_name] = note_text


    def setDelay(self, qtime):
        self.delay_time = datetime.timedelta(minutes=qtime.minute(), seconds=qtime.second())



    def onApplicationLaunched(self, app_name):
        if self.delay_time and isinstance(self.delay_time, datetime.timedelta):
            QTimer.singleShot(int(self.delay_time.total_seconds() * 1000), lambda: self.openReminderWindow(app_name))
        else:
            self.openReminderWindow(app_name)



    def openReminderWindow(self, app_name):
        note_text = self.notes.get(app_name, "")
        if note_text:
            self.ReminderWindow = ReminderWindow(note_text, self.geometry())
            self.ReminderWindow.setStyleSheet(self.currentTheme)
            self.ReminderWindow.show()

I also wanted to provide the option for darkmode within the app. It could use some improvement so feel free to contribute to the improvement of this specific section of the code.

In [ ]:

class ThemeColorButton(QPushButton):
    def __init__(self, parent=None, toggleFunction=None):
        super().__init__(parent)
        self.setMouseTracking(True)
        self.sunIcon = QIcon(icon_path('sun.png'))
        self.crescentIcon = QIcon(icon_path('crescent_moon.png'))
        self.toggleFunction = toggleFunction
        self.setIcon(self.crescentIcon)
        self.isDarkMode = False
        self.updateButtonStyle()
        
        
    def updateButtonStyle(self):
        
        self.setStyleSheet("QPushButton { border: none; }")


    def mousePressEvent(self, event):
        if self.toggleFunction:
            self.toggleFunction()  
        super().mousePressEvent(event)

I also believed that it would be necessary to include a help button for users to learn more about the application's features and what to be aware of for the future development of Event Notes.

In [ ]:
class HelpButton(QPushButton):
    def __init__(self, parent=None, helpFunction=None):
        super().__init__(parent)
        self.helpFunction = helpFunction
        self.setIcon(QIcon(icon_path('help_icon.png')))
        # self.setIconSize(QSize(24, 24))
        self.setStyleSheet("QPushButton { border: none; }")
        self.setToolTip("Help Information")
        self.clicked.connect(self.onClicked)
        

    def onClicked(self):
        if self.helpFunction:
            self.helpFunction()

For the application's primary function, it was necessary to create a compeletely seperate class for the calendar note window, due to the granular contributions that needed to be made to each part of the PyQt5 widgets and buttons.

In [ ]:
class CalendarNoteWindow(QWidget):
    def __init__(self, parent=None, callback=None, currentTheme=None):
        super().__init__(parent)
        self.setWindowFlags(Qt.Window)
        self.callback = callback
        self.currentTheme = currentTheme
        self.initUI()
        self.calendar.setMinimumDate(QDate.currentDate())


    def initUI(self):
        layout = QVBoxLayout(self)

        # Calendar Widget
        self.calendar = QCalendarWidget(self)
        self.calendar.setStyleSheet(CALENDAR_STYLESHEET)
        layout.addWidget(self.calendar)

        # Time Edit for setting specific time
        self.timeEdit = QTimeEdit(self)
        self.timeEdit.setDisplayFormat("HH:00:00")
        self.timeEdit.setTime(QTime(12, 0))  # Set initial time to 12:00
        layout.addWidget(self.timeEdit)


        # Text Edit for the note
        self.textEdit = QTextEdit()
        self.textEdit.setPlaceholderText("Enter your EventNote here...")
        layout.addWidget(self.textEdit)

        # Confirm Button
        self.confirmButton = QPushButton('Confirm Note', self)
        self.confirmButton.clicked.connect(self.confirmCalendarNote)
        sizePolicy = QSizePolicy(QSizePolicy.Minimum, QSizePolicy.Minimum)
        self.confirmButton.setSizePolicy(sizePolicy)
        self.confirmButton.setStyleSheet(BUTTON_STYLESHEET)
        layout.addWidget(self.confirmButton)

        self.setLayout(layout)
        self.setWindowTitle('Calendar Event Note')
        self.setWindowIcon(QIcon(icon_path('EventNotes_icon_48.png')))


    def createCustomMessageBox(self, title, title_icon, text, icon, buttons, defaultButton, customSize=None):
        msgBox = QMessageBox()
        msgBox.setWindowTitle(title)
        msgBox.setWindowIcon(title_icon)
        msgBox.setText(text)
        msgBox.setIcon(icon)
        msgBox.setStandardButtons(buttons)
        msgBox.setDefaultButton(defaultButton)
        msgBox.setStyleSheet(PROMPT_BUTTON_STYLESHEET)
        msgBox.setStyleSheet(self.currentTheme)
        if customSize:
            msgBox.setSizeGripEnabled(True)
            QSpacerItem = QtWidgets.QSpacerItem(customSize[0], customSize[1], QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding)
            layout = msgBox.layout()
            layout.addItem(QSpacerItem, layout.rowCount(), 0, 1, layout.columnCount())
        return msgBox  


    def confirmCalendarNote(self):
        note_text = self.textEdit.toPlainText()
        selected_date = self.calendar.selectedDate()
        selected_time = self.timeEdit.time()
        selected_datetime = datetime.datetime(
            selected_date.year(), selected_date.month(), selected_date.day(),
            selected_time.hour(), 0, 0
        )
    
        parent = self.parent()
        if isinstance(parent, EventNotes):
            # Check if there is an existing scheduled note for the selected date
            if parent.scheduledNote and parent.scheduledNote[0].date() == selected_date:
                # Show warning or replace the existing note
                msgBox = self.createCustomMessageBox(" ",
                                                     QIcon(icon_path('transparent_icon.png')),
                                                     "A note is already scheduled for this date. It will be replaced.",
                                                      QMessageBox.Warning,
                                                      QMessageBox.Ok,
                                                      QMessageBox.Ok)
                msgBox.exec_()
    
            if self.callback:
                self.callback(note_text, selected_datetime)
            self.close()
        else:
            print("Parent is not set or does not have a scheduledNote attribute.")

For convenience, I wanted to include a preferences window for the sake of the app's secondary functionary, and give users control over which buttons they would prefer to see represented in the main window of the application.

In [ ]:
class PreferencesWindow(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent, Qt.WindowCloseButtonHint)

        self.initUI()
        self.setFixedSize(200, self.sizeHint().height())
        

    def initUI(self):
        layout = QVBoxLayout(self)
        
        
        # Master Checkbox and Label
        self.selectAllCheckBox = QCheckBox("Select All", self)
        self.selectAllCheckBox.stateChanged.connect(self.selectAllChanged)
        layout.addWidget(self.selectAllCheckBox)

        
        checklistLabel = QLabel("Application Checklist:", self)
        layout.addWidget(checklistLabel)
        

        
        self.excelCheckBox = QCheckBox("Excel", self)
        layout.addWidget(self.excelCheckBox)

        
        self.powerPointCheckBox = QCheckBox("PowerPoint", self)
        layout.addWidget(self.powerPointCheckBox)
        
        
        self.wordCheckBox = QCheckBox("Word", self)
        layout.addWidget(self.wordCheckBox)
        
        
        self.outlookCheckBox = QCheckBox("Outlook", self)
        layout.addWidget(self.outlookCheckBox)
        
        
        
        self.teamsCheckBox = QCheckBox("Teams", self)
        layout.addWidget(self.teamsCheckBox)
        
        
        self.vscodeCheckBox = QCheckBox("Visual Studio Code", self)
        layout.addWidget(self.vscodeCheckBox)



        self.androidstudioCheckBox = QCheckBox("Android Studio", self)
        layout.addWidget(self.androidstudioCheckBox)
        
        
        self.dbeaverCheckBox = QCheckBox("Dbeaver", self)
        layout.addWidget(self.dbeaverCheckBox)
        
        
        
        self.chromeCheckBox = QCheckBox("Chrome", self)
        layout.addWidget(self.chromeCheckBox)
        
        
        self.edgeCheckBox = QCheckBox("Edge", self)
        layout.addWidget(self.edgeCheckBox)



        self.firefoxCheckBox = QCheckBox("Firefox", self)
        layout.addWidget(self.firefoxCheckBox)


        
        self.applyButton = QPushButton("Apply", self)
        self.applyButton.clicked.connect(self.accept)   
        self.applyButton.setStyleSheet(APPLY_BUTTON_STYLESHEET)
        layout.addWidget(self.applyButton)
        self.setLayout(layout)
        self.setWindowTitle('Preferences')
        self.setWindowIcon(QIcon(icon_path('preferences_icon_lm.png')))
    
    
    def selectAllChanged(self, state):
        # Set the checked state of all application checkboxes
        isChecked = state == Qt.Checked
        self.excelCheckBox.setChecked(isChecked)
        self.powerPointCheckBox.setChecked(isChecked)
        self.wordCheckBox.setChecked(isChecked)
        self.outlookCheckBox.setChecked(isChecked)
        self.teamsCheckBox.setChecked(isChecked)
        self.vscodeCheckBox.setChecked(isChecked)
        self.androidstudioCheckBox.setChecked(isChecked)
        self.dbeaverCheckBox.setChecked(isChecked)
        self.chromeCheckBox.setChecked(isChecked)
        self.edgeCheckBox.setChecked(isChecked)
        self.firefoxCheckBox.setChecked(isChecked)
            
        

    def getPreferences(self):
        return {
            'excel': self.excelCheckBox.isChecked(),
            'powerpoint': self.powerPointCheckBox.isChecked(),
            'word': self.wordCheckBox.isChecked(),
            'outlook': self.outlookCheckBox.isChecked(),
            'teams': self.teamsCheckBox.isChecked(),
            'vscode': self.vscodeCheckBox.isChecked(),
            'android studio': self.androidstudioCheckBox.isChecked(),
            'dbeaver': self.dbeaverCheckBox.isChecked(),
            'chrome': self.chromeCheckBox.isChecked(),
            'edge': self.edgeCheckBox.isChecked(),
            'firefox': self.firefoxCheckBox.isChecked()
        }

It was also necessary to create the universal notebook window for the notes created in association with the launch of a specific application within the Windows OS which I was able to achieve below:

In [ ]:
class ReminderWindow(QWidget):
    def __init__(self, note_text="", geometry=None):
        super().__init__()
        self.initUI(note_text, geometry)

    def initUI(self, note_text, geometry):
        self.setMinimumSize(500, 300)  
        layout = QVBoxLayout(self)
        label = QLabel(note_text, self)
        label.setAlignment(Qt.AlignCenter)
        
        label.setTextInteractionFlags(Qt.TextSelectableByMouse | Qt.TextSelectableByKeyboard)
        
        layout.addWidget(label)

        self.setLayout(layout)
        self.setWindowTitle('Event Notes')
        self.setWindowFlags(Qt.WindowStaysOnTopHint)  
        self.raise_()  
        self.activateWindow()

This is the end of the codebase for this project which has been distributed to the microsoft store under the terms of the GNU General Public License version 3 as published by the Free Software Foundation. If you have any specific questions or contributions, you are welcome to submit via this github repository page.